## Compiling Master Data Set

In [1]:
# Import packages
import pandas as pd
import seaborn as sns
import sqlite3 as sq
import matplotlib.pyplot as plt
%matplotlib inline

# Add extra relevant packages below as needed: 
import shutil
import numpy as np
import os
import re

In [2]:
# Import TMDB Database and format
db = pd.read_csv('zippedData/tmdb.movies.csv.gz')
db['genre_ids'].replace([''], np.nan, inplace=True)

#Import TN Database
tn_budget_raw = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

#Combine TN and TMDB databases
result = pd.merge(tn_budget_raw, db, left_on='movie', right_on='original_title',how='inner')
result.drop(columns=['id_x','Unnamed: 0','genre_ids','id_y','original_title','title'], inplace=True)

# Unzip IMDB SQL to new folder repository
import zipfile
with zipfile.ZipFile("zippedData/im.db.zip", 'r') as zip_ref:
    zip_ref.extractall("temp/IMDB_sql")
    
db_file = 'temp/IMDB_sql/im.db'

# Initializing imdb SQL database
# Create a SQL connection to our SQLite database
con = sq.connect(db_file)


## Query database for relevant information
genre_reviews = pd.read_sql(""" SELECT genres, primary_title, start_year
                
                FROM movie_basics
                LEFT JOIN movie_ratings ON movie_ratings.movie_id = movie_basics.movie_id
                JOIN movie_akas ON movie_akas.movie_id = movie_basics.movie_id
                WHERE start_year >= 2015 and region = 'US'
                ORDER BY averagerating DESC

""", con)

# Remove NaN values from SQL database
genre_reviews.dropna(inplace = True)

# Combine CSV and SQL data and format
comb_data = pd.merge(result, genre_reviews, left_on='movie', right_on='primary_title',how='inner')
comb_data.drop(columns=['primary_title', 'release_date_x','release_date_y'], inplace=True)
comb_data.drop_duplicates(subset=['movie'], inplace=True, ignore_index=True)

# Convert production budgets into ints
comb_data['production_budget'] = comb_data['production_budget'].str.strip('$')
comb_data['production_budget'] = comb_data['production_budget'].str.replace(',','')
comb_data['production_budget'] = comb_data['production_budget'].astype(int)

#Convert domestic gross into ints
comb_data['domestic_gross'] = comb_data['domestic_gross'].str.strip('$')
comb_data['domestic_gross'] = comb_data['domestic_gross'].str.replace(',','')
comb_data['domestic_gross'] = comb_data['domestic_gross'].astype(int)

# Convert worldwide gross into ints
comb_data['worldwide_gross'] = comb_data['worldwide_gross'].str.strip('$')
comb_data['worldwide_gross'] = comb_data['worldwide_gross'].str.replace(',','')
comb_data['worldwide_gross'] = comb_data['worldwide_gross'].astype(int)

# Removing movies that made less than $25,000
comb_data = comb_data[(comb_data['worldwide_gross'] > 25000)]

# Create movie net columns
comb_data['worldwide_net'] = comb_data['worldwide_gross'] - comb_data['production_budget']
comb_data['domestic_net'] = comb_data['domestic_gross'] - comb_data['production_budget']

# Sort by worldwide net
comb_data.sort_values(by="worldwide_net", ascending=False, inplace=True, ignore_index=True)

#remove temporary files
con.close()
shutil.rmtree('temp/')

In [3]:
# Create new df with genre counts higher than n
df1 = comb_data[comb_data['genres'].map(comb_data['genres'].value_counts()) > 15]

In [4]:
# Print master data frame
comb_data

,movie,production_budget,domestic_gross,worldwide_gross,original_language,popularity,vote_average,vote_count,genres,start_year,worldwide_net,domestic_net
0,Avengers: Infinity War,300000000,678815482,2048134200,en,80.773,8.3,13948,"Action,Adventure,Sci-Fi",2018,1748134200,378815482
1,Jurassic World,215000000,652270625,1648854864,en,20.709,6.6,14056,"Action,Adventure,Sci-Fi",2015,1433854864,437270625
2,Furious 7,190000000,353007020,1518722794,en,20.396,7.3,6538,"Action,Crime,Thriller",2015,1328722794,163007020
3,Black Panther,200000000,700059566,1348258224,en,2.058,5.1,11,"Action,Adventure,Sci-Fi",2018,1148258224,500059566
4,Jurassic World: Fallen Kingdom,170000000,417719760,1305772799,en,34.958,6.5,5995,"Action,Adventure,Sci-Fi",2018,1135772799,247719760
...,...,...,...,...,...,...,...,...,...,...,...,...
585,Turbulence,55000000,11532774,11532774,en,1.073,5.6,5,"Action,Drama,Thriller",2016,-43467226,-43467226
586,Blackhat,70000000,8005980,19665004,en,16.824,5.3,1165,"Action,Crime,Thriller",2015,-50334996,-61994020
587,Monster Trucks,125000000,33370166,61642798,en,8.827,5.9,414,"Action,Adventure,Comedy",2016,-63357202,-91629834
588,Air Strike,65000000,0,516279,en,6.971,3.9,56,"Action,Adventure,Drama",2018,-64483721,-65000000
